# 🏦 Investment Portfolio Review with Streaming Workflow

## Overview

This notebook demonstrates **streaming workflows** using Azure OpenAI Chat Agents for a **portfolio review** scenario. Watch real-time responses as two specialized agents analyze an investment portfolio.

### 💼 Industry Use Case: Portfolio Analysis Pipeline

A customer requests a review of their investment portfolio:
1. **Portfolio Analyst Agent** - Analyzes the portfolio composition and performance
2. **Risk Advisor Agent** - Evaluates risk factors and provides recommendations (streaming output)

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational and demonstration purposes only.** The investment analysis shown is simplified and should not be used for actual investment decisions. Always consult licensed financial advisors.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Azure OpenAI Integration** | Direct Azure OpenAI Service connectivity |
| **Real-time Streaming** | `AgentRunUpdateEvent` provides token-by-token updates |
| **output_response=True** | Marks terminal agent whose output is captured |
| **WorkflowBuilder** | Fluent API for agent workflow construction |

### Architecture

```
Customer Portfolio Request
    ↓
Portfolio Analyst Agent (composition analysis)
    ↓ streaming
Risk Advisor Agent (risk assessment & recommendations)
    ↓ streaming
Final Investment Guidance
```

## Prerequisites

- ✅ Azure OpenAI Service configured
- ✅ Environment variables: `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- ✅ Azure CLI authentication: Run `az login`

## 1️⃣ Setup and Imports

In [ ]:
import asyncio

from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


In [ ]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)
print("✅ Azure OpenAI Chat Client created")

## 2️⃣ Create Portfolio Analysis Agents

### 📊 Portfolio Analyst Agent
- Analyzes portfolio composition and allocation
- Evaluates diversification and performance
- First in the sequence

### ⚠️ Risk Advisor Agent
- Assesses risk factors based on analyst's findings
- Provides actionable recommendations
- Terminal agent with `output_response=True`

In [ ]:
# Portfolio Analyst Agent - Analyzes portfolio composition
portfolio_analyst = chat_client.as_agent(
    instructions=(
        "You are a Portfolio Analyst at an investment firm.\n"
        "When reviewing a portfolio:\n"
        "1. Analyze asset allocation (stocks, bonds, cash, alternatives)\n"
        "2. Evaluate sector diversification\n"
        "3. Note concentration risks\n"
        "4. Comment on overall portfolio balance\n"
        "Keep your analysis concise and data-driven."
    ),
    name="portfolio_analyst",
)
print("✅ Portfolio Analyst Agent created")

In [ ]:
# Risk Advisor Agent - Evaluates risk and provides recommendations
risk_advisor = chat_client.as_agent(
    instructions=(
        "You are a Risk Advisor at an investment firm.\n"
        "Based on the portfolio analysis:\n"
        "1. Assess overall risk level (Conservative/Moderate/Aggressive)\n"
        "2. Identify key risk factors\n"
        "3. Provide 2-3 specific recommendations to improve the portfolio\n"
        "4. Include standard investment disclaimers\n"
        "Be concise and actionable."
    ),
    name="risk_advisor",
)
print("✅ Risk Advisor Agent created")

## 3️⃣ Build Streaming Workflow

In [ ]:
# Build workflow: Portfolio Analyst → Risk Advisor
workflow = (
    WorkflowBuilder()
    .add_agent(portfolio_analyst, id="PortfolioAnalyst")
    .add_agent(risk_advisor, id="RiskAdvisor", output_response=True)
    .set_start_executor(portfolio_analyst)
    .add_edge(portfolio_analyst, risk_advisor)
    .build()
)
print("✅ Workflow built: Portfolio Analyst → Risk Advisor")

## 4️⃣ Submit Portfolio for Review with Streaming

Watch both agents analyze the portfolio in real-time!

In [ ]:
# Customer portfolio review request
portfolio_request = """
Please review my investment portfolio:

PORTFOLIO SUMMARY
=================
Total Value: $250,000

ASSET ALLOCATION:
- US Large Cap Stocks: 45% ($112,500)
- US Small Cap Stocks: 15% ($37,500)
- International Stocks: 10% ($25,000)
- Bonds: 20% ($50,000)
- Cash: 10% ($25,000)

TOP HOLDINGS:
1. Apple (AAPL) - 12%
2. Microsoft (MSFT) - 10%
3. Amazon (AMZN) - 8%
4. Nvidia (NVDA) - 7%
5. Tesla (TSLA) - 5%

INVESTOR PROFILE:
- Age: 35
- Risk Tolerance: Moderate
- Time Horizon: 20+ years (retirement)
"""

print("📊 PORTFOLIO REVIEW REQUEST")
print("=" * 60)
print(portfolio_request)
print("=" * 60 + "\n")

last_executor_id: str | None = None

events = workflow.run_stream(portfolio_request)
async for event in events:
    if isinstance(event, AgentRunUpdateEvent):
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            agent_emoji = "📊" if "Analyst" in eid else "⚠️"
            print(f"\n{agent_emoji} {eid}:", end=" ", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print("\n\n" + "=" * 60)
        print("📋 FINAL INVESTMENT GUIDANCE")
        print("=" * 60)
        print(event.data)

print("\n" + "=" * 60)
print("✅ Portfolio review complete!")
print("\n⚠️ DISCLAIMER: This analysis is for educational purposes only.")
print("   Always consult a licensed financial advisor for investment decisions.")